In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
df=pd.read_csv('Transaction_details.csv')
df

NameError: ignored

In [ ]:
pd.set_option('display.max_columns', 15)

pd.set_option('display.max_rows', 65535)

In [ ]:
df0 = pd.concat([df['TD_STRAN_REFERENCE'], df['TD_SCARD_BRAND'], df['TD_DTRAN_AMOUNT'], df['TD_STRAN_CURRENCY'],
                 df['TD_DTRAN_DATE_TIME'], df['TD_SINSTITUTION_ID'], df['TD_SMERCHANT_NAME'], df['TD_STERMINAL_ID'],
                 df['TD_SRESULT'], df['TD_SCARDHOLDER_NAME'],df['TD_SMASKEDCARDNUMBER'], df['TD_SCUST_IP'],
                 df['TD_SFRAUD_1_STATUS']], axis=1)

In [ ]:
df0.dtypes

TD_STRAN_REFERENCE       object
TD_SCARD_BRAND           object
TD_DTRAN_AMOUNT         float64
TD_STRAN_CURRENCY        object
TD_DTRAN_DATE_TIME       object
TD_SINSTITUTION_ID       object
TD_SMERCHANT_NAME        object
TD_STERMINAL_ID          object
TD_SRESULT               object
TD_SCARDHOLDER_NAME      object
TD_SMASKEDCARDNUMBER     object
TD_SCUST_IP              object
TD_SFRAUD_1_STATUS        int64
dtype: object

In [ ]:
null_counts = df0.isnull().sum()
null_counts

TD_STRAN_REFERENCE         43
TD_SCARD_BRAND           9322
TD_DTRAN_AMOUNT             0
TD_STRAN_CURRENCY           3
TD_DTRAN_DATE_TIME          0
TD_SINSTITUTION_ID         87
TD_SMERCHANT_NAME        2500
TD_STERMINAL_ID             0
TD_SRESULT                  6
TD_SCARDHOLDER_NAME      7759
TD_SMASKEDCARDNUMBER    13510
TD_SCUST_IP              4211
TD_SFRAUD_1_STATUS          0
dtype: int64

In [ ]:
columns_to_drop = null_counts[null_counts > 65532].index

df0 = df0.drop(columns_to_drop, axis=1)

In [ ]:
for column in df0.columns:
    # Check the data type of the column
    column_dtype = df0[column].dtype

    # Fill missing values based on the data type object as rest were filled
    if column_dtype == 'object':
        df0[column] = df0[column].fillna('Unknown')

# Verify if all missing values are filled
print(df0.isnull().sum())

TD_STRAN_REFERENCE      0
TD_SCARD_BRAND          0
TD_DTRAN_AMOUNT         0
TD_STRAN_CURRENCY       0
TD_DTRAN_DATE_TIME      0
TD_SINSTITUTION_ID      0
TD_SMERCHANT_NAME       0
TD_STERMINAL_ID         0
TD_SRESULT              0
TD_SCARDHOLDER_NAME     0
TD_SMASKEDCARDNUMBER    0
TD_SCUST_IP             0
TD_SFRAUD_1_STATUS      0
dtype: int64


In [ ]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

object_columns = df0.select_dtypes(include='object').columns

label_encoder = LabelEncoder()

for column in object_columns:
    df0[column] = df0[column].astype(str)
    df0[column] = label_encoder.fit_transform(df0[column])

In [ ]:
columns_to_scale = ['TD_DTRAN_AMOUNT']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df0[columns_to_scale] = scaler.fit_transform(df0[columns_to_scale])

In [ ]:
X = df0.drop('TD_SFRAUD_1_STATUS', axis=1)
y = df0['TD_SFRAUD_1_STATUS']

In [ ]:
from sklearn.model_selection import train_test_split
#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1434/1434 [==============================] - 3s 2ms/step - loss: 1.9004 - accuracy: 0.9893
Epoch 2/10
1434/1434 [==============================] - 2s 2ms/step - loss: 0.9176 - accuracy: 0.9900
Epoch 3/10
1434/1434 [==============================] - 2s 2ms/step - loss: 1.3521 - accuracy: 0.9898
Epoch 4/10
1434/1434 [==============================] - 3s 2ms/step - loss: 0.9880 - accuracy: 0.9902
Epoch 5/10
1434/1434 [==============================] - 2s 2ms/step - loss: 1.2250 - accuracy: 0.9904
Epoch 6/10
1434/1434 [==============================] - 2s 2ms/step - loss: 1.3962 - accuracy: 0.9903
Epoch 7/10
1434/1434 [==============================] - 2s 2ms/step - loss: 0.8389 - accuracy: 0.9904
Epoch 8/10
1434/1434 [==============================] - 2s 1ms/step - loss: 1.2208 - accuracy: 0.9901
Epoch 9/10
1434/1434 [==============================] - 2s 2ms/step - loss: 0.8288 - accuracy: 0.9904
Epoch 10/10
1434/1434 [==============================] - 3s 2ms/step - loss: 1.382

In [ ]:
model.evaluate(x_test, y_test)

615/615 [==============================] - 1s 1ms/step - loss: 1.7690 - accuracy: 0.9963


[1.769023060798645, 0.9962870478630066]

In [ ]:
y0 = model.predict(x_test)
y0

615/615 [==============================] - 1s 2ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
import numpy as np

y_pred = y0.astype(int)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19588
           1       0.00      0.00      0.00        73

    accuracy                           1.00     19661
   macro avg       0.50      0.50      0.50     19661
weighted avg       0.99      1.00      0.99     19661



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
